## SpotifyAPI Client
This notebook contains a fully documented copy of the SpotifyAPI Class.

In [1]:
import requests
import base64
import datetime
import json
from urllib.parse import urlencode
from typing import Union

In [16]:
class SpotifyAPI(object):
    """A class to make authorised api requests to Spotify.
    
    Can perform authentication to obtain an access token for the spotify api, and then make requests of that api
    such as searching for artist, album, or track data.
    
      Typical usage example:
      
      spotify = SpotifyAPI(client_id, client_secret)
      results = spotify.search(query)
    
    Attributes:
        access_token: a time-limited access token string provided by spotify for making api requests.
        access_token_expires: time at which current access_token expires.
        access_token_did_expire: expresses whether access_token has expired or not.
        client_id: spotify client id for the application.
        client_secret: spotify client secret for the application, relates to specific client id.
        token_url: url for obtaining spotify access token .
    """
    
    access_token = None
    access_token_expires = datetime.datetime.now
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        """Inits SpotifyAPI class performs authentication.
        
        Args:
            client_id (str): spotify client id for the application.
            client_secret (str): spotify client secret for the application, relates to specific client id.
        """
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.perform_auth()
       
    
    def get_client_credentials(self):
        """Combines client_id and client_secret and returns as a single base64 encoded string.
        
        Returns:
            client_creds_b64.decode(): client-id and client_secret combined as a base64 encoded string.
        """
        
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_secret == None or client_id == None:
            raise Exception("client_id and client_secret required.")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    
    def get_token_headers(self):
        """Returns headers for access token api request.
        
        Returns:
            A dict containing a formatted string for access token requests contining client credential information.
        """
        
        client_creds_b64 = self.get_client_credentials()
        return { 
        "Authorization" : f"Basic {client_creds_b64}"
        }
    
    
    def get_token_data(self):
        """Creates token data as a dict.
                
        Returns:
            A dict containing access token request data, authorisation granted based on client credential information.
        """
        
        return {
        "grant_type" : "client_credentials"
        }
     
        
    def perform_auth(self):
        """Performs authentication for making api requests to Spotify.
        
        Returns:
            True once authentication has been successfully completed.
            
        Raises:
            Exception: Authentication failed - ensure client credentials are correct.
        """
        
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception("Authentication failed - ensure client credentials are correct.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        """Obtains and returns a valid access token.
        
        Returns:
            token(str): access token for making authenticated api requests to Spotify.
        """
        
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    
    def get_resource_headers(self):
        """Returns headers for resource api request.
        
        Returns:
            headers(dict): a dict containing access token for making  authorised api requests.
        """
        access_token = self.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}
        return headers
    
    
    def get_resource(self, lookup_id, resource_type="track", version="v1"):
        """Makes an api request for resources from spotify.
        
        Retrieves JSON data related to resource request type and returns as a dict of raw 
        unaltered data.
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to a specific object 
              such as artist, track, album etc.
            resource_type(str): type of resource that relates to lookup_id such as 'track', 'artist', 
              'album' etc.
           version(str): a string relating to version of spotify api. At point of creation only v1 is 
             available.
        
        Returns:
            r.json() (dict): a dict containing raw unaltered data related to the requested resource. If 
              request is nsuccessful, returns an empty dict.  
        """
        
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_headers()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    
    def get_artist(self, lookup_id):
        """Gets artist data from spotify based on lookup_id
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to specific artist, 
              usually obtained from SpotifyAPI.search().
        
        Returns:
            A dict containing raw unaltered data related to the artist. If request is 
              unsuccessful, returns an empty dict.
        """
        
        return self.get_resource(lookup_id, resource_type="artists")
    
    
    def get_album(self, lookup_id):
        """Gets album data from spotify based on lookup_id
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to specific album, 
              usually obtained from SpotifyAPI.search().
        
        Returns:
            A dict containing raw unaltered data related to the album. If request is 
              unsuccessful, returns an empty dict.
        """
        
        return self.get_resource(lookup_id, resource_type="albums")
    
    
    def get_track_features(self, lookup_id):
        """Gets album data from spotify based on lookup_id
        
        Args:
            lookup_id(str): a string of 22 alphanumeric characters related to specific album, 
              usually obtained from SpotifyAPI.search().
        
        Returns:
            A dict containing raw unaltered data related to the album. If request is 
              unsuccessful, returns an empty dict.
        """
        
        return self.get_resource(lookup_id, resource_type="audio-features")

    
    def base_search(self, query_params: str) -> dict:
        """TODO:
        
        Makes a search api request for resources from spotify.
        
        Takes in url query string (typically from self.search), gets the headers
        for spotify api queries, and returns search data as dict. 
        """
        lookup_url = f"https://api.spotify.com/v1/search?{query_params}"
        headers = self.get_resource_headers()
        
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        if not r.status_code in range(200, 299):
            return {}
        return r.json()
    
    
    def search(self, query: Union[str,dict]=None, search_type: str="artist", market_type: str="GB") -> dict:
        """TODO:
        
        Makes a search api request for resources from spotify. 
        
        Takes in url query string (typically from self.search), gets the headers
        for spotify api queries, and returns search data as dict. 
        """
        
        new_query=""
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            for key,value in query.items():
                new_query += f"{key}:{value} "
        query_params = urlencode({"q" : new_query, "type" : search_type.lower(), "market" : market_type})
        return self.base_search(query_params)
   
    
    def get_tracks(self, query: Union[str,dict]=None) -> list:
        """TODO:
        
        Finds track information and returns as a list of dicts.
        """
        
        json_data = self.search(query, "track")
        try: 
            tracks = [{'track_id': i['id'], 
              'track_name': i['name'],
              'artist': i['artists'][0]['name'],
              'track_url': i['external_urls']['spotify'],
              'image_url': i['album']['images'][0]['url']
             } for i in json_data['tracks']['items']]
        except:
            printf("json_data dict not valid")
            return [] 
        return tracks
    
    
    def get_musical_data(self, track_id: str) -> dict:
        """TODO:
        
        Takes in a single track_id and returns a dict containing
        track_id, with formatted song key and song tempo information.
        """
        
        try:
            track_data = self.get_track_features(track_id)
            key = self.key_convert(track_data['key'], track_data['mode'])
            tempo = int(track_data['tempo'])
            musical_data = {"track_id": track_id,
                            "key": key,
                            "tempo": tempo}
            return musical_data
        except:
            raise Exception("No data found")
        
    
    def key_convert(self, key: int, mode: int=None) -> str:
        """TODO:
        
        """
        
        if key < 0 or key > 10:
            raise Exception("Invalid key value entered - bad Python joke")
        keys = {0: "C", 1: "Db", 2: "D", 3: "Eb",
                4: "Eb", 5: "F", 6: "Gb", 7: "G",
                8: "Ab", 9: "B", 10: "Bb"}
        if mode == None:
            return keys[key]
        elif mode == 1:
            return f"{keys[key]} Major"
        else:
            return f"{keys[key]} Minor"

        
    def time_signature_convert(self, time_signature: int) -> str:
        """TODO:
        
        """
        
        if time_signature < 0 or time_signature > 10:
            return "invalid key"
        keys = {0: "C", 1: "Db", 2: "D", 3: "Eb",
                4: "Eb", 5: "F", 6: "Gb", 7: "G",
                8: "Ab", 9: "B", 10: "Bb"}
        return keys[key]
        

### SEARCH
Let's try searching for something!

In [19]:
client_id = "a3aadf7f4d184af0af9d849d96e89b12"
client_secret = ""

In [20]:
# Create spotify Api instance
spotify = SpotifyAPI(client_id, client_secret)
# Search for tracks using multiple query arguements.
spotify.get_tracks({"track": "money", "artist": "pink floyd"})

https://api.spotify.com/v1/search?q=track%3Amoney+artist%3Apink+floyd+&type=track&market=GB
200


[{'track_id': '7Gx2q0ueNwvDp2BOZYGCMO',
  'track_name': 'Money - 2011 Remastered Version',
  'artist': 'Pink Floyd',
  'track_url': 'https://open.spotify.com/track/7Gx2q0ueNwvDp2BOZYGCMO',
  'image_url': 'https://i.scdn.co/image/ab67616d0000b27331c57b302f0e3aca46ab7561'},
 {'track_id': '6FjDEJNmx4XWYilqhTs7G9',
  'track_name': 'Money - Live',
  'artist': 'Pink Floyd',
  'track_url': 'https://open.spotify.com/track/6FjDEJNmx4XWYilqhTs7G9',
  'image_url': 'https://i.scdn.co/image/ab67616d0000b27369f4b7cda08f4ed73cc20474'},
 {'track_id': '77IH1EBjOSSBo30ie5XwEo',
  'track_name': 'Money - Live',
  'artist': 'Pink Floyd',
  'track_url': 'https://open.spotify.com/track/77IH1EBjOSSBo30ie5XwEo',
  'image_url': 'https://i.scdn.co/image/ab67616d0000b2737946212e06e309d833b7999e'},
 {'track_id': '1WakuskS7SyaWnXqU4cQS7',
  'track_name': 'Money - 2001 Remastered Version',
  'artist': 'Pink Floyd',
  'track_url': 'https://open.spotify.com/track/1WakuskS7SyaWnXqU4cQS7',
  'image_url': 'https://i.scdn

In [7]:
# Get formatted musical data based on track id
spotify.get_musical_data("7Gx2q0ueNwvDp2BOZYGCMO")

{'track_id': '7Gx2q0ueNwvDp2BOZYGCMO', 'key': 'B Major', 'tempo': 124}

In [21]:
item = spotify.get_access_token()
type(item)

str